In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
df = pd.read_csv('./preprocessed_twitter_data.csv')

In [3]:
negative_tweets = df[df['sentiment'] == 0].head(500000)
print(negative_tweets.shape)
print(negative_tweets)

(500000, 3)
        Unnamed: 0                                              tweet  \
0                0  upset updat facebook text might cri result sch...   
1                1       dive mani time ball manag save rest go bound   
2                2                    whole bodi feel itchi like fire   
3                3                                      behav mad see   
4                4                                         whole crew   
...            ...                                                ...   
499995      499995                      idea use web p tweetdeck work   
499996      499996          ww work gain back probabl put chip anyway   
499997      499997                                   sorri mommi miss   
499998      499998     terribl headach last night weight decreas aaah   
499999      499999                                         cant sleep   

        sentiment  
0               0  
1               0  
2               0  
3               0  
4          

In [4]:
positive_tweets = df[df['sentiment'] == 1].head(500000)
print(positive_tweets.shape)
print(positive_tweets)

(500000, 3)
         Unnamed: 0                                              tweet  \
799999       799999                                  love u guy r best   
800000       800000      im meet one besti tonight cant wait girl talk   
800001       800001  thank twitter add sunisa got meet hin show dc ...   
800002       800002  sick realli cheap hurt much eat real food plu ...   
800003       800003                                     effect everyon   
...             ...                                                ...   
1299994     1299994                      watch horton hear tasha apart   
1299995     1299995      r u friendfe got friend request u sure realli   
1299996     1299996  thank comment grace record toast ice americano...   
1299997     1299997  look launch uniqu art web store uniqu art shir...   
1299998     1299998  new blog potenti cake diaster ahhhh www mandic...   

         sentiment  
799999           1  
800000           1  
800001           1  
800002         

In [5]:
neutral_tweets = df[df['sentiment'] == 2].head(10000)
print(neutral_tweets.shape)
print(neutral_tweets)

(10000, 3)
         Unnamed: 0                                              tweet  \
3123980     3123980                              top tablet damn right   
3123981     3123981              cnbctv appl margin better expect aapl   
3123987     3123987  rt bought store pretti good logo match wait in...   
3124000     3124002  latest appl product lead effici iphon ipad plu...   
3124005     3124007                              rt thank think upgrad   
...             ...                                                ...   
3141591     3141673   better anyon els busi stock invest tradeidea gnu   
3141592     3141674  rt sound eerili like famou julian robertson un...   
3141593     3141675  arent extrem optimist yet market go spx spi iw...   
3141594     3141676  case covid florida past week still set open pa...   
3141595     3141677  investor take chanc biogen stock biib rhhbi nv...   

         sentiment  
3123980          2  
3123981          2  
3123987          2  
3124000         

In [6]:
from sklearn.model_selection import train_test_split

# Split negative_tweets into train and test sets
train_negative_tweets, test_negative_tweets = train_test_split(negative_tweets, test_size=0.3, random_state=42)

# Split positive_tweets into train and test sets
train_positive_tweets, test_positive_tweets = train_test_split(positive_tweets, test_size=0.3, random_state=42)

# Split neutral_tweets into train and test sets
train_neutral_tweets, test_neutral_tweets = train_test_split(neutral_tweets, test_size=0.3, random_state=42)

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.utils import shuffle

# Combine the train sets for all sentiment categories and shuffle
train_tweets = shuffle(pd.concat([train_negative_tweets, train_positive_tweets, train_neutral_tweets], ignore_index=True), random_state=42)

# Combine the test sets for all sentiment categories and shuffle
test_tweets = shuffle(pd.concat([test_negative_tweets, test_positive_tweets, test_neutral_tweets], ignore_index=True), random_state=42)

# Handle NaN values in the 'text' column
train_tweets = train_tweets.dropna(subset=['tweet'])
test_tweets = test_tweets.dropna(subset=['tweet'])

# Get the normalized text reviews from the combined train and test sets
tweet_train_reviews = train_tweets['tweet'].tolist()
tweet_test_reviews = test_tweets['tweet'].tolist()

# Train lables and test lables of tweets
train_labels = train_tweets['sentiment']
test_labels = test_tweets['sentiment']

# Initialize the tokenizer
tokenizer = Tokenizer()

# Fit the tokenizer on the training data
tokenizer.fit_on_texts(tweet_train_reviews)

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(tweet_train_reviews)
test_sequences = tokenizer.texts_to_sequences(tweet_test_reviews)

# Ensure sequences are padded to the same length
maxlen = max(len(seq) for seq in train_sequences + test_sequences)
print("Maxlen: " , maxlen)
train_sequences = pad_sequences(train_sequences, maxlen=maxlen)
test_sequences = pad_sequences(test_sequences, maxlen=maxlen)


Maxlen:  56


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Assuming you have already defined vocab_size, embedding_dim, maxlen, train_sequences, and train_labels
# ...
vocab_size = len(tokenizer.word_index) + 1  # add 1 for the padding token
embedding_dim = 100  # adjust as needed
maxlen = maxlen  # as defined earlier
train_sequences = train_sequences  # as defined earlier
train_labels = train_labels  # as defined earlier

# Define the model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen))
model.add(LSTM(units=100))
model.add(Dense(units=3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Train the model
model.fit(train_sequences, train_labels, epochs=5, batch_size=64, validation_split=0.2)
model.evaluate(test_sequences, test_labels)




Epoch 1/5


8797/8797 [==============================] - 1192s 135ms/step - loss: 0.4915 - accuracy: 0.7662 - val_loss: 0.4720 - val_accuracy: 0.7756
Epoch 2/5
8797/8797 [==============================] - 952s 108ms/step - loss: 0.4275 - accuracy: 0.8037 - val_loss: 0.4731 - val_accuracy: 0.7764
Epoch 3/5
8797/8797 [==============================] - 954s 108ms/step - loss: 0.3745 - accuracy: 0.8322 - val_loss: 0.4999 - val_accuracy: 0.7690
Epoch 4/5
8797/8797 [==============================] - 954s 108ms/step - loss: 0.3290 - accuracy: 0.8540 - val_loss: 0.5303 - val_accuracy: 0.7645
Epoch 5/5
9424/9424 [==============================] - 62s 7ms/step - loss: 0.5789 - accuracy: 0.7580


[0.5788764953613281, 0.757967472076416]

In [9]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)